# Audio Effects Processing with SoundLab

This notebook demonstrates SoundLab's audio effects processing powered by Pedalboard.

**What you'll learn:**
- Build effects chains
- Apply dynamics processing (compressor, limiter, gate)
- Use EQ filters (highpass, lowpass, parametric)
- Add time-based effects (reverb, delay, chorus)
- Apply creative effects (distortion, phaser)
- Process audio files and arrays
- Best practices for effects processing

## Setup

Import necessary modules and configure the environment.

In [ ]:
import soundlab
from soundlab.effects import EffectsChain
from soundlab.effects.models import (
    # Dynamics
    CompressorConfig, LimiterConfig, GateConfig, GainConfig,
    # EQ
    HighpassConfig, LowpassConfig, HighShelfConfig, LowShelfConfig, PeakFilterConfig,
    # Time-based
    ReverbConfig, DelayConfig, ChorusConfig, PhaserConfig,
    # Creative
    DistortionConfig, ClippingConfig,
)
from soundlab.io import load_audio, save_audio
from pathlib import Path

# For audio playback
from IPython.display import Audio, display
import numpy as np

print(f"SoundLab version: {soundlab.__version__}")

## 1. Basic Effects Chain

Let's start by creating a simple effects chain:

In [ ]:
# Path to your audio file
input_file = "../../tests/fixtures/audio/music_like_5s.wav"
output_dir = Path("./output/effects")
output_dir.mkdir(parents=True, exist_ok=True)

# Listen to the original
print("Original audio:")
display(Audio(input_file))

# Create an effects chain
chain = EffectsChain()

# Add effects (fluent API)
chain.add(HighpassConfig(cutoff_hz=80.0))  # Remove low rumble
chain.add(CompressorConfig(threshold_db=-20.0, ratio=4.0))  # Compress dynamics
chain.add(ReverbConfig(room_size=0.5, wet_level=0.3))  # Add space

print(f"\nEffects chain: {chain}")
print(f"Number of effects: {len(chain)}")

# Process the audio
output_file = output_dir / "basic_chain.wav"
chain.process(input_file, output_file)

print(f"\nProcessed audio saved to: {output_file}")
print("\nProcessed audio:")
display(Audio(str(output_file)))

## 2. Dynamics Processing

Control the dynamic range of audio with compression, limiting, and gating:

### Compression

Reduce dynamic range by attenuating loud signals:

In [ ]:
# Create compressor configurations
compressors = {
    "gentle": CompressorConfig(
        threshold_db=-20.0,
        ratio=2.0,
        attack_ms=10.0,
        release_ms=100.0,
    ),
    "medium": CompressorConfig(
        threshold_db=-15.0,
        ratio=4.0,
        attack_ms=5.0,
        release_ms=100.0,
    ),
    "heavy": CompressorConfig(
        threshold_db=-10.0,
        ratio=8.0,
        attack_ms=1.0,
        release_ms=50.0,
    ),
}

print("Compressor Settings:\n")
for name, config in compressors.items():
    print(f"{name.capitalize()} compression:")
    print(f"  Threshold: {config.threshold_db} dB")
    print(f"  Ratio: {config.ratio}:1")
    print(f"  Attack: {config.attack_ms} ms")
    print(f"  Release: {config.release_ms} ms\n")

# Apply different compression levels
for name, config in compressors.items():
    chain = EffectsChain().add(config)
    output = output_dir / f"compressed_{name}.wav"
    chain.process(input_file, output)
    print(f"Saved {name} compression: {output.name}")

### Limiter

Prevent peaks from exceeding a threshold:

In [ ]:
# Limiter for peak control
limiter_chain = EffectsChain()
limiter_chain.add(LimiterConfig(
    threshold_db=-1.0,  # Prevent peaks above -1 dB
    release_ms=100.0,
))

# Add makeup gain and limit
mastering_chain = EffectsChain()
mastering_chain.add(GainConfig(gain_db=3.0))  # Increase volume
mastering_chain.add(LimiterConfig(threshold_db=-0.5, release_ms=50.0))  # Prevent clipping

output = output_dir / "limited.wav"
mastering_chain.process(input_file, output)

print(f"Mastering chain: {mastering_chain}")
print(f"Output: {output.name}")
print("\nLimited audio:")
display(Audio(str(output)))

### Noise Gate

Attenuate signals below a threshold to reduce noise:

In [ ]:
# Noise gate to clean up quiet sections
gate_chain = EffectsChain()
gate_chain.add(GateConfig(
    threshold_db=-40.0,  # Gate opens above -40 dB
    ratio=10.0,          # Strong attenuation below threshold
    attack_ms=1.0,       # Fast attack
    release_ms=100.0,    # Smooth release
))

output = output_dir / "gated.wav"
gate_chain.process(input_file, output)

print(f"Noise gate applied: {output.name}")

## 3. EQ and Filtering

Shape the frequency spectrum with various filters:

### Basic Filters (Highpass/Lowpass)

In [ ]:
# Highpass filter - remove low frequencies
highpass_chain = EffectsChain()
highpass_chain.add(HighpassConfig(cutoff_hz=100.0))  # Remove below 100 Hz

output = output_dir / "highpass.wav"
highpass_chain.process(input_file, output)
print(f"Highpass filtered: {output.name}")

# Lowpass filter - remove high frequencies
lowpass_chain = EffectsChain()
lowpass_chain.add(LowpassConfig(cutoff_hz=8000.0))  # Remove above 8 kHz

output = output_dir / "lowpass.wav"
lowpass_chain.process(input_file, output)
print(f"Lowpass filtered: {output.name}")

# Band-pass (combine high and low pass)
bandpass_chain = EffectsChain()
bandpass_chain.add(HighpassConfig(cutoff_hz=200.0))
bandpass_chain.add(LowpassConfig(cutoff_hz=5000.0))

output = output_dir / "bandpass.wav"
bandpass_chain.process(input_file, output)
print(f"Bandpass filtered (200 Hz - 5 kHz): {output.name}")

### Shelving Filters

In [ ]:
# Boost highs, cut lows
shelf_chain = EffectsChain()
shelf_chain.add(LowShelfConfig(
    cutoff_hz=200.0,
    gain_db=-3.0,  # Cut bass by 3 dB
    q=0.707,
))
shelf_chain.add(HighShelfConfig(
    cutoff_hz=4000.0,
    gain_db=2.0,   # Boost treble by 2 dB
    q=0.707,
))

output = output_dir / "shelved.wav"
shelf_chain.process(input_file, output)

print(f"Shelving EQ: {shelf_chain}")
print(f"Output: {output.name}")
print("\nShelved audio:")
display(Audio(str(output)))

### Parametric EQ (Peak Filters)

In [ ]:
# Multi-band parametric EQ
eq_chain = EffectsChain()

# Cut muddy low-mids
eq_chain.add(PeakFilterConfig(
    cutoff_hz=250.0,
    gain_db=-2.0,
    q=1.0,
))

# Boost presence
eq_chain.add(PeakFilterConfig(
    cutoff_hz=3000.0,
    gain_db=3.0,
    q=1.5,
))

# Cut harshness
eq_chain.add(PeakFilterConfig(
    cutoff_hz=8000.0,
    gain_db=-1.5,
    q=2.0,
))

output = output_dir / "eq.wav"
eq_chain.process(input_file, output)

print("Parametric EQ Chain:")
print(f"  {eq_chain}")
print(f"\nOutput: {output.name}")
print("\nEQ'd audio:")
display(Audio(str(output)))

## 4. Time-Based Effects

Add depth and space with reverb, delay, and modulation:

### Reverb

In [ ]:
# Different reverb styles
reverbs = {
    "small_room": ReverbConfig(
        room_size=0.3,
        damping=0.7,
        wet_level=0.2,
        dry_level=0.8,
        width=0.5,
    ),
    "medium_hall": ReverbConfig(
        room_size=0.6,
        damping=0.5,
        wet_level=0.3,
        dry_level=0.7,
        width=1.0,
    ),
    "large_cathedral": ReverbConfig(
        room_size=0.9,
        damping=0.3,
        wet_level=0.4,
        dry_level=0.6,
        width=1.0,
    ),
}

print("Reverb Presets:\n")
for name, config in reverbs.items():
    print(f"{name.replace('_', ' ').title()}:")
    print(f"  Room size: {config.room_size}")
    print(f"  Damping: {config.damping}")
    print(f"  Wet/Dry: {config.wet_level}/{config.dry_level}\n")
    
    # Process
    chain = EffectsChain().add(config)
    output = output_dir / f"reverb_{name}.wav"
    chain.process(input_file, output)
    print(f"  Saved: {output.name}\n")

### Delay

In [ ]:
# Delay effects
delays = {
    "slapback": DelayConfig(
        delay_seconds=0.1,  # 100ms
        feedback=0.0,
        mix=0.3,
    ),
    "echo": DelayConfig(
        delay_seconds=0.375,  # Dotted eighth at 120 BPM
        feedback=0.4,
        mix=0.3,
    ),
    "long_echo": DelayConfig(
        delay_seconds=0.75,
        feedback=0.5,
        mix=0.4,
    ),
}

print("Delay Effects:\n")
for name, config in delays.items():
    print(f"{name.replace('_', ' ').title()}:")
    print(f"  Delay: {config.delay_seconds * 1000:.0f} ms")
    print(f"  Feedback: {config.feedback}")
    print(f"  Mix: {config.mix}")
    
    chain = EffectsChain().add(config)
    output = output_dir / f"delay_{name}.wav"
    chain.process(input_file, output)
    print(f"  Saved: {output.name}\n")

### Modulation (Chorus, Phaser)

In [ ]:
# Chorus effect
chorus_chain = EffectsChain()
chorus_chain.add(ChorusConfig(
    rate_hz=1.0,
    depth=0.3,
    centre_delay_ms=7.0,
    feedback=0.2,
    mix=0.5,
))

output = output_dir / "chorus.wav"
chorus_chain.process(input_file, output)
print(f"Chorus: {output.name}")
print("\nChorus audio:")
display(Audio(str(output)))

# Phaser effect
phaser_chain = EffectsChain()
phaser_chain.add(PhaserConfig(
    rate_hz=0.5,
    depth=0.5,
    centre_frequency_hz=1300.0,
    feedback=0.3,
    mix=0.5,
))

output = output_dir / "phaser.wav"
phaser_chain.process(input_file, output)
print(f"\nPhaser: {output.name}")
print("\nPhaser audio:")
display(Audio(str(output)))

## 5. Creative Effects

Add character with distortion and clipping:

In [ ]:
# Subtle distortion
distortion_chain = EffectsChain()
distortion_chain.add(DistortionConfig(drive_db=10.0))

output = output_dir / "distorted.wav"
distortion_chain.process(input_file, output)
print(f"Distortion: {output.name}")

# Hard clipping
clipping_chain = EffectsChain()
clipping_chain.add(ClippingConfig(threshold_db=-6.0))

output = output_dir / "clipped.wav"
clipping_chain.process(input_file, output)
print(f"Clipping: {output.name}")

## 6. Complex Effects Chains

Combine multiple effects for professional processing:

### Vocal Processing Chain

In [ ]:
vocal_chain = EffectsChain()

# 1. Clean up
vocal_chain.add(HighpassConfig(cutoff_hz=80.0))  # Remove rumble
vocal_chain.add(GateConfig(threshold_db=-45.0, ratio=10.0))  # Remove noise

# 2. EQ for clarity
vocal_chain.add(PeakFilterConfig(cutoff_hz=250.0, gain_db=-2.0, q=1.0))  # Cut mud
vocal_chain.add(PeakFilterConfig(cutoff_hz=3000.0, gain_db=3.0, q=1.5))  # Boost presence
vocal_chain.add(HighShelfConfig(cutoff_hz=8000.0, gain_db=2.0, q=0.707))  # Air

# 3. Dynamics control
vocal_chain.add(CompressorConfig(
    threshold_db=-18.0,
    ratio=3.0,
    attack_ms=5.0,
    release_ms=100.0,
))

# 4. Add space
vocal_chain.add(ReverbConfig(
    room_size=0.4,
    damping=0.6,
    wet_level=0.15,
    dry_level=0.85,
))

# 5. Safety limiter
vocal_chain.add(LimiterConfig(threshold_db=-1.0, release_ms=100.0))

print("Vocal Processing Chain:")
print(f"  {vocal_chain}")
print(f"\nEffects: {vocal_chain.effect_names}")

output = output_dir / "vocal_processed.wav"
vocal_chain.process(input_file, output)
print(f"\nOutput: {output.name}")
print("\nProcessed vocal:")
display(Audio(str(output)))

### Mastering Chain

In [ ]:
mastering_chain = EffectsChain()

# 1. Surgical EQ
mastering_chain.add(HighpassConfig(cutoff_hz=30.0))  # Remove subsonic
mastering_chain.add(PeakFilterConfig(cutoff_hz=150.0, gain_db=-1.0, q=0.7))  # Tame bass
mastering_chain.add(HighShelfConfig(cutoff_hz=10000.0, gain_db=0.5, q=0.707))  # Subtle air

# 2. Gentle compression
mastering_chain.add(CompressorConfig(
    threshold_db=-15.0,
    ratio=1.5,
    attack_ms=30.0,
    release_ms=200.0,
))

# 3. Limiting for loudness
mastering_chain.add(LimiterConfig(threshold_db=-0.3, release_ms=50.0))

print("Mastering Chain:")
print(f"  {mastering_chain}")
print(f"\nEffects: {mastering_chain.effect_names}")

output = output_dir / "mastered.wav"
mastering_chain.process(input_file, output)
print(f"\nOutput: {output.name}")
print("\nMastered audio:")
display(Audio(str(output)))

### Creative Lofi Chain

In [ ]:
lofi_chain = EffectsChain()

# 1. Reduce fidelity
lofi_chain.add(HighpassConfig(cutoff_hz=200.0))
lofi_chain.add(LowpassConfig(cutoff_hz=6000.0))

# 2. Add character
lofi_chain.add(DistortionConfig(drive_db=8.0))

# 3. Modulation for wobble
lofi_chain.add(ChorusConfig(
    rate_hz=0.3,
    depth=0.2,
    centre_delay_ms=5.0,
    mix=0.3,
))

# 4. Vinyl-like reverb
lofi_chain.add(ReverbConfig(
    room_size=0.3,
    damping=0.8,
    wet_level=0.2,
    dry_level=0.8,
))

print("Lofi Processing Chain:")
print(f"  {lofi_chain}")
print(f"\nEffects: {lofi_chain.effect_names}")

output = output_dir / "lofi.wav"
lofi_chain.process(input_file, output)
print(f"\nOutput: {output.name}")
print("\nLofi audio:")
display(Audio(str(output)))

## 7. Working with Audio Arrays

Process audio arrays directly for more control:

In [ ]:
# Load audio
audio, sample_rate = load_audio(input_file)

print(f"Audio shape: {audio.shape}")
print(f"Sample rate: {sample_rate} Hz")
print(f"Duration: {len(audio) / sample_rate:.2f} seconds")

# Create effects chain
chain = EffectsChain()
chain.add(CompressorConfig(threshold_db=-20.0, ratio=4.0))
chain.add(ReverbConfig(room_size=0.5, wet_level=0.3))

# Ensure correct shape for processing (channels, samples)
if audio.ndim == 1:
    audio_2d = audio.reshape(1, -1)
else:
    audio_2d = audio

# Process array
processed = chain.process_array(audio_2d.astype(np.float32), sample_rate)

print(f"\nProcessed shape: {processed.shape}")

# Convert back to 1D if needed
if processed.shape[0] == 1:
    processed_1d = processed[0]
else:
    processed_1d = processed

# Save
output = output_dir / "array_processed.wav"
save_audio(output, processed_1d, sample_rate)

print(f"Saved: {output.name}")

## 8. Chain Management

Manipulate effects chains dynamically:

In [ ]:
# Create a chain
chain = EffectsChain()
chain.add(HighpassConfig(cutoff_hz=100.0))
chain.add(CompressorConfig(threshold_db=-20.0, ratio=4.0))
chain.add(ReverbConfig(room_size=0.5, wet_level=0.3))

print("Initial chain:")
print(f"  {chain}")
print(f"  Length: {len(chain)}")
print(f"  Effects: {chain.effect_names}")

# Insert an effect
chain.insert(1, GainConfig(gain_db=2.0))
print("\nAfter inserting gain at position 1:")
print(f"  {chain}")

# Remove an effect
chain.remove(2)  # Remove compressor
print("\nAfter removing effect at position 2:")
print(f"  {chain}")

# Clear all effects
chain.clear()
print("\nAfter clearing:")
print(f"  {chain}")
print(f"  Length: {len(chain)}")

### Creating Chains from Configs

In [ ]:
# Create a list of effect configs
configs = [
    HighpassConfig(cutoff_hz=80.0),
    CompressorConfig(threshold_db=-20.0, ratio=4.0),
    ReverbConfig(room_size=0.5, wet_level=0.3),
    LimiterConfig(threshold_db=-1.0),
]

# Create chain from configs
chain = EffectsChain.from_configs(configs)

print("Chain created from configs:")
print(f"  {chain}")
print(f"  Effects: {chain.effect_names}")

## 9. A/B Comparison

Compare original and processed audio:

In [ ]:
import matplotlib.pyplot as plt

def compare_audio(original_file, processed_file):
    """
    Compare original and processed audio visually.
    """
    # Load both files
    original, sr_orig = load_audio(original_file)
    processed, sr_proc = load_audio(processed_file)
    
    # Create time axes
    time_orig = np.arange(len(original)) / sr_orig
    time_proc = np.arange(len(processed)) / sr_proc
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    # Original
    ax1.plot(time_orig, original, linewidth=0.5, alpha=0.7, color='steelblue')
    ax1.set_ylabel('Amplitude', fontsize=10)
    ax1.set_title('Original', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-1, 1)
    
    # Processed
    ax2.plot(time_proc, processed, linewidth=0.5, alpha=0.7, color='crimson')
    ax2.set_xlabel('Time (seconds)', fontsize=10)
    ax2.set_ylabel('Amplitude', fontsize=10)
    ax2.set_title('Processed', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-1, 1)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate RMS levels
    rms_orig = np.sqrt(np.mean(original**2))
    rms_proc = np.sqrt(np.mean(processed**2))
    
    print(f"\nRMS Levels:")
    print(f"  Original: {20 * np.log10(rms_orig):.2f} dBFS")
    print(f"  Processed: {20 * np.log10(rms_proc):.2f} dBFS")
    print(f"  Difference: {20 * np.log10(rms_proc/rms_orig):.2f} dB")

# Compare
processed_file = output_dir / "mastered.wav"
if processed_file.exists():
    compare_audio(input_file, processed_file)
    
    print("\nListen to both:")
    print("\nOriginal:")
    display(Audio(input_file))
    print("\nProcessed:")
    display(Audio(str(processed_file)))

## Best Practices and Tips

### 1. Effect Order Matters

**Typical signal flow:**
1. **Corrective EQ** (highpass, problem frequency cuts)
2. **Dynamics** (gate, compressor)
3. **Creative EQ** (tone shaping, boosts)
4. **Saturation/Distortion**
5. **Modulation** (chorus, phaser)
6. **Time-based** (delay, reverb)
7. **Limiting** (final peak control)

### 2. Dynamics Processing

**Compressor:**
- Lower threshold, lower ratio: Transparent control
- Higher threshold, higher ratio: Obvious compression
- Fast attack: Controls transients
- Slow attack: Preserves punch
- Fast release: Pumping effect
- Slow release: Smooth, natural

**Limiter:**
- Use as safety net, not main dynamics control
- Leave 0.5-1.0 dB headroom for encoding
- Faster release for electronic music
- Slower release for acoustic music

**Gate:**
- Set threshold just above noise floor
- Use fast attack (1-5 ms)
- Adjust release to avoid cutting off natural decay

### 3. EQ Guidelines

**General principles:**
- Cut before boost
- Use narrow Q for cuts, wide Q for boosts
- Highpass almost everything (except kick, bass)
- Be subtle with boosts (1-3 dB usually enough)

**Frequency ranges:**
- Sub-bass: 20-60 Hz (feel rather than hear)
- Bass: 60-250 Hz (weight and power)
- Low-mids: 250-500 Hz (warmth, can be muddy)
- Mids: 500-2000 Hz (body, presence)
- Upper-mids: 2-5 kHz (clarity, definition)
- Highs: 5-10 kHz (brightness, air)
- Ultra-highs: 10-20 kHz (sparkle, air)

### 4. Reverb and Delay

**Reverb:**
- Less is more - start with low wet levels
- Increase damping for natural sound
- Match room size to genre (small for intimate, large for epic)
- Use pre-delay to maintain clarity

**Delay:**
- Sync to tempo for musical delays
- Common times: 1/4, 1/8, dotted 1/8
- Lower feedback for subtle effects
- Higher feedback for dub/psychedelic effects

### 5. Common Mistakes

- **Over-processing**: Less is often more
- **Too much compression**: Kills dynamics
- **Excessive EQ**: Creates phase issues
- **Too much reverb**: Makes mix muddy
- **Ignoring gain staging**: Causes clipping or noise
- **Processing in wrong order**: Suboptimal results

### 6. Genre-Specific Tips

**Electronic/EDM:**
- Heavy compression/limiting for loudness
- Sidechain compression for pumping
- Saturation for warmth

**Rock:**
- Preserve dynamics
- Use parallel compression
- Emphasize mids for guitars

**Acoustic/Folk:**
- Minimal processing
- Gentle compression
- Natural reverb

**Hip-Hop:**
- Strong low-end
- Vocal clarity (3-5 kHz boost)
- Punchy drums (fast attack on compressor)

### 7. Monitoring

- A/B compare with original frequently
- Check on multiple playback systems
- Take breaks to avoid ear fatigue
- Reference professional tracks
- Check in mono for phase issues

## Summary

In this notebook, you learned how to:

✓ Build and manage effects chains  
✓ Apply dynamics processing (compression, limiting, gating)  
✓ Use EQ and filters (highpass, lowpass, parametric)  
✓ Add time-based effects (reverb, delay, modulation)  
✓ Apply creative effects (distortion, clipping)  
✓ Create complex processing chains  
✓ Process audio files and arrays  
✓ Compare original and processed audio  
✓ Apply best practices for effects processing  

**Next Steps:**
- Combine effects with stem separation
- Create genre-specific processing chains
- Experiment with creative effects combinations
- Build your own processing templates